# ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [ ]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [ ]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    file_path_list = glob.glob(os.path.join(root,'*'))

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [ ]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [ ]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

## The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Apache Cassandra code in the cells below

#### Creating a Cluster

In [ ]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [ ]:
# Cassandra Keyspace 
session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }
""")

#### Set Keyspace

In [ ]:
# Set KEYSPACE to the keyspace specified above
session.set_keyspace('sparkify')

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## The following three questions are needed to be extracted from of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [ ]:
# The query needs 5 columns, artist, song title, song's length, sessionId and itemInSessionId. The WHERE clause requires the sessionId and the itemInSession, which indicates that these two columns should conform the primary key. The order in which they appeare in the WHERE clause is also important.
query1 = """
    SELECT artist, song_title, duration FROM session_events WHERE session_id = 338 AND item_in_session = 4
"""

# This leads to the following SQL table definition:
session.execute("""
    CREATE TABLE IF NOT EXISTS session_events (
        session_id int,
        item_in_session int,
        artist text,
        song_title text,
        duration float,
        PRIMARY KEY(session_id, item_in_session)
    )
""")

In [ ]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO session_events (session_id, item_in_session, artist, song_title, duration)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

In [ ]:
rows = session.execute(query1)

df = pd.DataFrame(list(rows))
df.head()

In [ ]:
# The query needs 7 columns, artist, song title, first name, last name, user id, sessionId and itemInSessionId. The WHERE clause requires the user id and the sessionId, which indicates that these two columns should conform the primary key, but also the partition key, to make sure that session information for a single user gets stored in the same node within the cluster. It is also important to note that the results should be sorted by itemInSession, i.e. the column should be included as a clustering column in the primary key.
query2 = """
    SELECT artist, song_title, first_name, last_name FROM user_session_events WHERE user_id = 10 AND session_id = 182
"""

# This leads to the following SQL table definition:
session.execute("""
    CREATE TABLE IF NOT EXISTS user_session_events (
        user_id int,
        session_id int,
        item_in_session int,
        artist text,
        song_title text,
        first_name text,
        last_name text,
        PRIMARY KEY((user_id, session_id), item_in_session)
    )
""")

In [ ]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_session_events (user_id, session_id, item_in_session, artist, song_title, first_name, last_name)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

In [ ]:
rows = session.execute(query2)

df = pd.DataFrame(list(rows))
df.head()

In [ ]:
# The query needs 3 columns, song title, first name and last name. The WHERE clause requires the song title, so it should be included in the primary key, but since there might be multiple users that listen to that song, the user id is also required in the primary key to avoid duplicates.
query3 = """
    SELECT first_name, last_name FROM user_events WHERE song_title = 'All Hands Against His Own'
"""

# This leads to the following SQL table definition:
session.execute("""
    CREATE TABLE IF NOT EXISTS user_events (
        song_title text,
        user_id int,
        first_name text,
        last_name text,
        PRIMARY KEY(song_title, user_id)
    )
""")

In [ ]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_events (song_title, user_id, first_name, last_name)"
        query = query + "VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

In [ ]:
rows = session.execute(query3)

df = pd.DataFrame(list(rows))
df.head()

### Drop the tables before closing out the sessions

In [ ]:
session.execute("""DROP TABLE IF EXISTS session_events""")
session.execute("""DROP TABLE IF EXISTS user_session_events""")
session.execute("""DROP TABLE IF EXISTS user_events""")

### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()